In [1]:
from utils import processed, load, denormalize, get_normalized_adj, generate_dataset
import torch

In [2]:
num_timesteps_input = 6 # 30 minutes
num_timesteps_output = 2 # 10 minutes

In [3]:
raw_trunc_dir = "./data/raw/trunc/"
process_dir = "./data/processed/"

# overwrite = False means that the processing function will only run if the process data files do not exist
# overwrite = True => functions will run regardless
processed(raw_trunc_dir, process_dir, overwrite=False)
A, X, metadata, means, stds = load(process_dir)

split_line1 = int(X.shape[2] * 0.6)
split_line2 = int(X.shape[2] * 0.8)

train_original_data = X[:, :, :split_line1]
val_original_data = X[:, :, split_line1:split_line2]
test_original_data = X[:, :, split_line2:]

training_input, training_target = generate_dataset(train_original_data,
                                                   num_timesteps_input=num_timesteps_input,
                                                   num_timesteps_output=num_timesteps_output)
val_input, val_target = generate_dataset(val_original_data,
                                         num_timesteps_input=num_timesteps_input,
                                         num_timesteps_output=num_timesteps_output)
test_input, test_target = generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)
# input shape (num_samples ,num_vertices, num_timesteps_window, num_features)
# output shape (num_samples ,num_vertices, num_timesteps_window)
A_wave = get_normalized_adj(A)
A_wave = torch.from_numpy(A_wave)

In [4]:
print(f"training input shape: {training_input.shape} training target shape: {training_target.shape}")
print(f"val input shape: {val_input.shape} val target shape: {val_target.shape}")
print(f"test input shape: {test_input.shape} test target shape: {test_target.shape}")
print(f"Adjacency matrix shape: {A.shape}")
print(f"Adjacency matrix shape: {A_wave.shape}")

training input shape: torch.Size([4491, 53, 6, 3]) training target shape: torch.Size([4491, 53, 2])
val input shape: torch.Size([1492, 53, 6, 3]) val target shape: torch.Size([1492, 53, 2])
test input shape: torch.Size([1493, 53, 6, 3]) test target shape: torch.Size([1493, 53, 2])
Adjacency matrix shape: (53, 53)
Adjacency matrix shape: torch.Size([53, 53])


In [5]:
print(metadata)
print(means)
print(stds)

{'B': 0, 'C': 1}
[3.993683   0.9056604  0.05673373]
[1.0052049  0.29230058 0.03545753]


In [6]:
# how to denormalize the speedband
denormalize(training_target, stds[0], means[0])

tensor([[[4., 4.],
         [5., 4.],
         [4., 4.],
         ...,
         [4., 5.],
         [4., 4.],
         [4., 5.]],

        [[4., 4.],
         [4., 4.],
         [4., 5.],
         ...,
         [5., 5.],
         [4., 4.],
         [5., 5.]],

        [[4., 4.],
         [4., 4.],
         [5., 5.],
         ...,
         [5., 4.],
         [4., 4.],
         [5., 4.]],

        ...,

        [[3., 3.],
         [3., 3.],
         [4., 4.],
         ...,
         [4., 4.],
         [3., 3.],
         [5., 5.]],

        [[3., 3.],
         [3., 3.],
         [4., 4.],
         ...,
         [4., 4.],
         [3., 3.],
         [5., 5.]],

        [[3., 3.],
         [3., 3.],
         [4., 6.],
         ...,
         [4., 4.],
         [3., 3.],
         [5., 5.]]])